-sandbox

<div style="text-align: center; line-height: 0; padding-top: 9px;">
  <img src="https://databricks.com/wp-content/uploads/2018/03/db-academy-rgb-1200px.png" alt="Databricks Learning" style="width: 600px">
</div>

# Type 2 Slowly Changing Data

In this notebook, we'll create a silver table that contains the information we'll need to link workouts back to our heart rate recordings.

We'll use a Type 2 table to record this data, encoding the start and end times for each session. 

<img src="https://files.training.databricks.com/images/ade/ADE_arch_completed_workouts.png" width="60%" />

## Learning Objectives
By the end of this lesson, students will be able to:
- Describe how Slowly Changing Dimension tables can be implemented in the Lakehouse
- Use custom logic to implement a SCD Type 2 table with batch overwrite logic

## Setup
Set up path and checkpoint variables (these will be used later).

In [0]:
%run ../Includes/Classroom-Setup-4.4

Creating the database "dbacademy_odl_user_771624_databrickslabs_com_adewd_4_4"

Creating date_lookup...(4 seconds / 1,096 records)

Loading batch #1 to the daily stream...(8 seconds, 417,276 records)
Processing the bronze table from the daily stream...(14 seconds / 417,276 records)

Predefined Paths:
 DA.paths.working_dir: dbfs:/user/odl_user_771624@databrickslabs.com/dbacademy/adewd/4.4
 DA.paths.user_db: dbfs:/user/odl_user_771624@databrickslabs.com/dbacademy/adewd/4.4/4_4.db
 DA.paths.checkpoints: dbfs:/user/odl_user_771624@databrickslabs.com/dbacademy/adewd/4.4/_checkpoints
 DA.paths.source_daily: dbfs:/user/odl_user_771624@databrickslabs.com/dbacademy/adewd/4.4/streams/daily

Predefined tables in dbacademy_odl_user_771624_databrickslabs_com_adewd_4_4:
 bronze
 date_lookup

Validating datasets...(1 seconds)

Setup completed in 28 seconds

## Review workouts_silver Table
Several helper functions was defined to land and propagate a batch of data to the **`workouts_silver`** table.

This table is created by 
* Starting a stream against the **`bronze`** table
* Filtering all records by **`topic = 'workout'`**
* Deduping the data 
* Merging non-matching records into **`owrkouts_silver`**

...roughly the same strategy we used earlier to create the **`heart_rate_silver`** table

In [0]:
DA.daily_stream.load()       # Load another day's data
DA.process_bronze()          # Update the bronze table
DA.process_workouts_silver() # Update the workouts_silver table

Loading batch #2 to the daily stream...(8 seconds, 462,415 records)
Processing the bronze table from the daily stream...(7 seconds / 879,691 records)
Processing the workouts_silver table...(11 seconds / 152 records)

Review the **`workouts_silver`** data.

In [0]:
workout_df = spark.read.table("workouts_silver")
display(workout_df)

user_id,workout_id,time,action,session_id
24863,47,2019-12-02T15:06:40.000+0000,stop,2
36117,5,2019-12-02T07:38:40.000+0000,stop,314
38766,30,2019-12-02T07:34:24.000+0000,start,402
16093,42,2019-12-02T14:53:52.000+0000,stop,493
40872,37,2019-12-02T14:58:08.000+0000,stop,78
37012,40,2019-12-02T17:38:08.000+0000,start,55
30088,6,2019-12-02T17:27:28.000+0000,start,445
26285,19,2019-12-02T15:57:52.000+0000,stop,73
45875,20,2019-12-02T14:28:16.000+0000,start,210
26285,19,2019-12-02T14:32:32.000+0000,start,73


For this data, the **`user_id`** and **`session_id`** form a composite key. 

Each pair should eventually have 2 records present, marking the "start" and "stop" action for each workout.

In [0]:
aggregate_df = workout_df.groupby("user_id", "session_id").count()
display(aggregate_df)

user_id,session_id,count
16093,493,2
33987,487,2
19198,407,2
27703,456,2
12474,2,2
28588,1,2
45875,207,2
24018,148,2
16093,492,2
28776,390,2


Because we'll be triggering a shuffle in this notebook, we'll be explicit about how many partitions we want at the end of our shuffle.

As before, we can use the current level of parallelism (max number of cores) as our upper bound for shuffle partitions.

In [0]:
print(f"Executor cores: {sc.defaultParallelism}")
spark.conf.set("spark.sql.shuffle.partitions", sc.defaultParallelism)

Executor cores: 4

## Create Completed Workouts Table

The query below matches our start and stop actions, capturing the time for each action. The **`in_progress`** field indicates whether or not a given workout session is ongoing.

In [0]:
def process_completed_workouts():
    spark.sql(f"""
        CREATE OR REPLACE TABLE completed_workouts 
        AS (
          SELECT a.user_id, a.workout_id, a.session_id, a.start_time start_time, b.end_time end_time, a.in_progress AND (b.in_progress IS NULL) in_progress
          FROM (
            SELECT user_id, workout_id, session_id, time start_time, null end_time, true in_progress
            FROM workouts_silver
            WHERE action = "start") a
          LEFT JOIN (
            SELECT user_id, workout_id, session_id, null start_time, time end_time, false in_progress
            FROM workouts_silver
            WHERE action = "stop") b
          ON a.user_id = b.user_id AND a.session_id = b.session_id
        )
    """)
    
process_completed_workouts()

You can now perform a query directly on your **`completed_workouts`** table to check your results.

In [0]:
total = spark.table("completed_workouts").count() # .sql("SELECT COUNT(*) FROM completed_workouts") 
print(f"{total:3} total")

total = spark.table("completed_workouts").filter("in_progress=true").count()
print(f"{total:3} where record is still awaiting end time")

total = spark.table("completed_workouts").filter("end_time IS NOT NULL").count()
print(f"{total:3} where end time has been recorded")

total = spark.table("completed_workouts").filter("start_time IS NOT NULL").count()
print(f"{total:3} where end time arrived after start time")

total = spark.table("completed_workouts").filter("in_progress=true AND end_time IS NULL").count()
print(f"{total:3} where they are in_progress AND have an end_time")

76 total
 0 where record is still awaiting end time
 76 where end time has been recorded
 76 where end time arrived after start time
 0 where they are in_progress AND have an end_time

Use the functions below to propagate another batch of records through the pipeline to this point.

In [0]:
DA.daily_stream.load()       # Load another day's data
DA.process_bronze()          # Update the bronze table
DA.process_workouts_silver() # Update the workouts_silver table

process_completed_workouts() # Update the completed_workouts table

Loading batch #3 to the daily stream...(7 seconds, 496,291 records)
Processing the bronze table from the daily stream...(7 seconds / 1,375,982 records)
Processing the workouts_silver table...(8 seconds / 238 records)

In [0]:
%sql
SELECT COUNT(*) 
AS total 
FROM completed_workouts

total
119


Run the following cell to delete the tables and files associated with this lesson.

In [0]:
DA.cleanup()

Dropping the database "dbacademy_odl_user_771624_databrickslabs_com_adewd_4_4"
Removing the working directory "dbfs:/user/odl_user_771624@databrickslabs.com/dbacademy/adewd/4.4"
Validating datasets...(1 seconds)

-sandbox
&copy; 2022 Databricks, Inc. All rights reserved.<br/>
Apache, Apache Spark, Spark and the Spark logo are trademarks of the <a href="https://www.apache.org/">Apache Software Foundation</a>.<br/>
<br/>
<a href="https://databricks.com/privacy-policy">Privacy Policy</a> | <a href="https://databricks.com/terms-of-use">Terms of Use</a> | <a href="https://help.databricks.com/">Support</a>